In [1]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))

In [2]:
# Path to dataset
dataset_folder = ' ' # FILL OUT FIRST !!!

In [3]:
paths = DisplayablePath.make_tree(Path(dataset_folder))

for path in paths:
    print(path.displayable())

us_presidential_speeches/
├── democrat/
│   ├── Barack Obama/
│   ├── Bill Clinton/
│   ├── Harry Truman/
│   │   ├── Address at the National Archives Dedicating the New Shrine for the Declaration of Independence.txt
│   │   ├── Address Before the National Association for the Advancement of Colored People.txt
│   │   ├── Address in Philadelphia Upon Accepting the Nomination of the Democratic National Convention.txt
│   │   ├── April 11, 1951 - Report to the American People on Korea.txt
│   │   ├── April 16, 1945 - First Speech to Congress.txt
│   │   ├── August 6, 1945 - Statement by the President Announcing the Use of the A-Bomb at Hiroshima.txt
│   │   ├── August 9, 1945 - Radio Report to the American People on the Potsdam Conference.txt
│   │   ├── Farewell Address.txt
│   │   ├── Inaugural Address.txt
│   │   ├── June 20, 1947 - On the Veto of the Taft-Hartley Bill.txt
│   │   ├── March 15, 1952 - Columbia Scholastic Press Association.txt
│   │   ├── March 29, 1952 - Jefferson-Jack